In [38]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import os 
import gzip
import random

MNIST = input_data.read_data_sets('MNIST_data/', one_hot = True)

# x = tf.placeholder('float', [None, 784])
# W = tf.Variable(tf.zeros([784,10]))
# b = tf.Variable(tf.zeros([10]))
# y = tf.nn.softmax(tf.matmul(x,W) + b)

# y_ = tf.placeholder('float', [None, 10])
# cross_entropy = -tf.reduce_sum(y_*tf.log(y))
# step_train = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = 'float'))
# # 初始化变量
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(1000):
#         batch_xs, batch_ys =  MNIST.train.next_batch(100)
#         sess.run(step_train, feed_dict = {x:batch_xs, y_:batch_ys})
#         if (i+1) % 100 ==0:
#             print ('Step:', i, '====accuracy:',sess.run(accuracy, {x:MNIST.test.images, y_:MNIST.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [47]:
# tf.constant?
# with tf.Session() as sess:
#     print (sess.run(tf.Variable(tf.ones([5,5,1,1]))))

In [39]:
# 构建多层卷积网络
# 参数初始化
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
# 卷积和池化
def conv2d(x, W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1,], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [36]:
# with tf.Session() as sess:
#     print (sess.run(tf.truncated_normal([5, 5, 1, 24],stddev=0.1)))
#     print (tf.truncated_normal([5, 5, 1, 24],stddev=0.1))
#     print (sess.run(tf.constant(0.1, shape = [6])))

In [44]:
# 第一层卷积
W_conv1 = weight_variable([5,5,1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1 标识任意数量的样本数,大小为28*28,深度为1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 第二层卷积
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 密集连接层
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 输出层|
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [46]:
# 训练&评估
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        batch =  MNIST.train.next_batch(50)
#         print (type(batch))
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:MNIST.tes, y_: batch[1], keep_prob: 1.0})
            print ("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    # print ("test accuracy %g"%accuracy.eval(feed_dict={x: MNIST.test.images, y_: MNIST.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.9
step 200, training accuracy 0.84
step 300, training accuracy 0.86
step 400, training accuracy 0.9
step 500, training accuracy 0.88
step 600, training accuracy 1
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.92
step 1100, training accuracy 0.96
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.98
step 1500, training accuracy 0.96
step 1600, training accuracy 1
step 1700, training accuracy 0.94
step 1800, training accuracy 0.94
step 1900, training accuracy 0.96
step 2000, training accuracy 1
step 2100, training accuracy 1
step 2200, training accuracy 0.94
step 2300, training accuracy 0.98
step 2400, training accuracy 0.98
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, train